In [56]:
import json

with open('result_dict.json', 'r') as file:
    data = json.load(file)

# Now you can work with the data from the JSON file

In [57]:
from utils import create_dictionary
newdict = create_dictionary(data)

In [58]:
from utils import write_dataset

dataset = write_dataset(newdict)

In [59]:
dataset[0]

{'id': '0',
 'tokens': ['Teacher:',
  'Patient',
  'présente',
  'avec',
  'un',
  'épisode',
  'affectif',
  'mixte,',
  'caractérisé',
  'par',
  'des',
  'périodes',
  'de',
  'manie',
  'et',
  'dépression',
  'sévères',
  'en',
  'alternance.'],
 'ner_tags': [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [60]:
len(dataset)

2063

In [61]:
total_len = len(dataset)

# Define the split sizes
train_size = 1444
remaining_size = total_len - train_size
validation_size = remaining_size // 2
test_size = remaining_size - validation_size

# Split the data into train, validation, and test sets
train_data = dataset[:train_size]
validation_data = dataset[train_size:train_size + validation_size]
test_data = dataset[train_size + validation_size:]

In [62]:
from datasets import Dataset, DatasetDict, ClassLabel, Sequence, Value
import pandas as pd






# Define features for the datasets


# Create Dataset objects
train_dataset = Dataset.from_pandas(pd.DataFrame(data=train_data))
validation_dataset = Dataset.from_pandas(pd.DataFrame(data=validation_data))
test_dataset = Dataset.from_pandas(pd.DataFrame(data=test_data))


from transformers import AutoTokenizer

model_checkpoint = "distilbert/distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

label_names = ['O', 'Maladie']

# Create ClassLabel feature
classmap = ClassLabel(num_classes = 2, names=label_names)

train_dataset = train_dataset.map(lambda x: tokenizer(x["tokens"], truncation=True))
validation_dataset = validation_dataset.map(lambda x: tokenizer(x["tokens"], truncation=True))
test_dataset = test_dataset.map(lambda x: tokenizer(x["tokens"], truncation=True))

# Map labels to label ids.
train_dataset = train_dataset.map(lambda y: {"labels": classmap.str2int(y["ner_tags"])})
validation_dataset = validation_dataset.map(lambda y: {"labels": classmap.str2int(y["ner_tags"])})
test_dataset = test_dataset.map(lambda y: {"labels": classmap.str2int(y["ner_tags"])})



Map: 100%|██████████| 310/310 [00:00<00:00, 15077.63 examples/s]


In [63]:
train_dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1444
})

In [64]:
# Create a DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

# Display the DatasetDict
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1444
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 309
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 310
    })
})


In [65]:
from datasets import ClassLabel

# Example label names
label_names = ['O', 'Maladie']

# Define ClassLabel feature
class_label = ClassLabel(names=label_names)

# Ensure ner_tags are valid
for entry in dataset_dict["test"]:
    for tag in entry['ner_tags']:
        if tag >= len(label_names):
            print(f"Invalid tag {tag} in entry {entry['id']}")

# Création du tokenizer

In [66]:
dataset_dict["train"]["ner_tags"]

[[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [67]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert/distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [68]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            
            new_labels.append(label)

    return new_labels

In [69]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [70]:
tokenized_datasets = dataset_dict.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_dict["train"].column_names,
)

Map: 100%|██████████| 310/310 [00:00<00:00, 17800.70 examples/s]


In [71]:
tokenized_datasets["train"][0]

{'input_ids': [101,
  72451,
  131,
  24714,
  15617,
  19446,
  10460,
  10119,
  17809,
  60600,
  13918,
  82649,
  117,
  15903,
  14526,
  71377,
  18255,
  10248,
  10139,
  108384,
  10104,
  52321,
  10112,
  10131,
  97184,
  71779,
  15492,
  10477,
  16401,
  10110,
  42141,
  71252,
  119,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -100]}

In [72]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [73]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [74]:
import evaluate

metric = evaluate.load("seqeval")

In [75]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [76]:
label2id

{'O': 0, 'Maladie': 1}

In [77]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [78]:
train_dataloader.dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1444
})

In [79]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [80]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

Problèmes avec les étiquettes de classe : Les étiquettes utilisées pour l'entraînement peuvent être hors des limites définies (par exemple, une étiquette ner_tags qui n'est pas définie dans ClassLabel).

Problèmes de données d'entrée : Les données d'entrée peuvent être mal formatées ou contenir des erreurs.

Incompatibilité de la configuration CUDA : Le GPU peut ne pas être correctement configuré pour la tâche.

In [81]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [82]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [83]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner"
repo_name = get_full_repo_name(model_name)
repo_name

'Amhyr/bert-finetuned-ner'

In [84]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

Cloning https://huggingface.co/Amhyr/bert-finetuned-ner into local empty directory.






















































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [85]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [86]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

 33%|███▎      | 181/543 [14:18<28:36,  4.74s/it]


epoch 0: {'precision': 0.5344827586206896, 'recall': 0.4444444444444444, 'f1': 0.48532289628180036, 'accuracy': 0.9519367701509541}


epoch 1: {'precision': 0.603448275862069, 'recall': 0.5017921146953405, 'f1': 0.547945205479452, 'accuracy': 0.9547137567644546}


epoch 2: {'precision': 0.5732758620689655, 'recall': 0.4907749077490775, 'f1': 0.5288270377733598, 'accuracy': 0.9561378524636855}
